In [1]:
from __future__ import print_function
import argparse
import os
import random
import numpy as np
import torch as torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from pointnet import PointNetDenseCls
import torch.nn.functional as F
import matplotlib.pyplot as plt
from show3d_balls import *
from dataset_provider import Dataset

In [2]:
d = Dataset('./dataset', './tool_categories.txt', 2500, 'knife')

In [ ]:
# visualizing segmentation 

# model number which is used -> has to be the same as seg_model number (so if idx 2 then seg_model_2.pth)
idx = 2

print("model %d/%d" %( idx, len(d)))

point, seg = d[idx]

print(point.size(), seg.size())

point_np = point.numpy()

cmap = plt.cm.get_cmap("hsv", 10)
cmap = np.array([cmap(i) for i in range(10)])[:,:3]
gt = cmap[seg.numpy() - 1, :]


""" k is number of categories in our case 7
  1 - 'grasp'
  2 - 'cut'
  3 - 'scoop'
  4 - 'contain'
  5 - 'pound'
  6 - 'support'
  7 - 'wrap-grasp'
"""
classifier = PointNetDenseCls(k = 7)
classifier.load_state_dict(torch.load('seg/seg_model_2.pth'))
classifier.eval()

point = point.transpose(1,0).contiguous()

point = Variable(point.view(1, point.size()[0], point.size()[1]))
pred, _ = classifier(point)
pred_choice = pred.data.max(2)[1]
print(pred_choice)

#print(pred_choice.size())
pred_color = cmap[pred_choice.numpy()[0], :]

#print(pred_color.shape)
showpoints(point_np, gt, pred_color)

model 2/513
torch.Size([2500, 3]) torch.Size([2500])
tensor([[0, 0, 0,  ..., 0, 0, 0]])
